In [ ]:
!pip install transformers
!pip install huggingface_hub
!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token(<YOUR_TOKEN_HERE>)"

In [ ]:
!pip install sentencepiece
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!pip install gradio

In [ ]:
import requests
from transformers import pipeline
import nltk
from nltk import sent_tokenize
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import pipeline

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="SnypzZz/Llama2-13b-Language-translate", use_fast = False)

In [ ]:
tokenizer = MBart50TokenizerFast.from_pretrained("SnypzZz/Llama2-13b-Language-translate", src_lang="en_XX")
model = None
model_loaded = False

In [ ]:
api_token_header = ""
with open('/content/secrets.py', 'r') as f:
  api_token_header = f.read()

In [ ]:
def load_model():
    global model, model_loaded
    model = MBartForConditionalGeneration.from_pretrained("SnypzZz/Llama2-13b-Language-translate")
    model_loaded =True
    return model

In [ ]:
def translation(text,dest_lang,dest_lang_code, src_lang_code):

    if(dest_lang_code == src_lang_code):
        return "Please select different languages to translate between."

    # headers = {"Authorization": f"Bearer {secrets_sih.api_token_header}"}
    headers = {"Authorization": f"Bearer {api_token_header}"}

    # Bengali Done
    if(dest_lang == "Bengali" and src_lang_code == "en_XX"):
        API_URL = "https://api-inference.huggingface.co/models/csebuetnlp/banglat5_nmt_en_bn"
        def query(payload):
            response = requests.post(API_URL, headers=headers, json=payload)
            return response.json()
        output = query({
            "inputs": text,
        })
        print(output)
        return output[0]['translation_text']
    else:
        global model
        if model:
            pass
        else:
            model = load_model()
        loaded_model = model
        tokenizer = MBart50TokenizerFast.from_pretrained("SnypzZz/Llama2-13b-Language-translate", src_lang=src_lang_code)
        #model_inputs = tokenizer(text, return_tensors="pt")
        loaded_model_inputs = tokenizer(text, return_tensors="pt")

        # translate
        generated_tokens = loaded_model.generate(
            **loaded_model_inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id[dest_lang_code]
        )
        output = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        print(output)
        return output[0]

In [ ]:
def main_translation(text,dest_lang_code,src_lang_code):

    codes = {"en_XX":"English","bn_IN":"Bengali", "en_GB":"English","gu_IN":"Gujarati","hi_IN":"Hindi","ta_IN":"Tamil","te_IN":"Telugu","mr_IN":"Marathi"}
    dest_lang = codes[dest_lang_code]
    src_lang = codes[src_lang_code]

    sentences = sent_tokenize(text)
    output = ""
    for line in sentences:
        output += translation(line,dest_lang,dest_lang_code, src_lang_code)
    return {"output":output}

In [ ]:
main_translation("Hello World", "hi_IN","en_XX")

['हेलो वर्ल्ड']


{'output': 'हेलो वर्ल्ड'}

In [ ]:
#User Interface

import gradio as gr


def test(text, src, dest):
  ans = main_translation(text,dest,src)
  return ans['output']
demo = gr.Interface(
    test,
    ["textbox",
     gr.Dropdown(
            [("English", "en_XX"), ("Hindi","hi_IN"), ("Bengali","bn_IN"), ("Gujarati","gu_IN"), ("Tamil","ta_IN"), ("Telugu","te_IN"), ("Marathi","mr_IN")], label="Source", info="Select the Source Language!"
        ),
     gr.Dropdown(
            [("English", "en_XX"), ("Hindi","hi_IN"), ("Bengali","bn_IN"), ("Gujarati","gu_IN"), ("Tamil","ta_IN"), ("Telugu","te_IN"), ("Marathi","mr_IN")], label="Destination", info="Select the Destination Language!"
        ),
     ],
    outputs=["textbox"],
)

demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3800d81e3201b8d734.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
